Train MLP for each finger pair!\
Using hyper param optimalization
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [3]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne
from random import shuffle
import random

mne.set_log_level('error')


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [4]:
# Set seed for NumPy
np.random.seed(42)

# Set seed for Python's built-in random number generator
random.seed(42)

# Set seed for PyTorch
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
subject_id = 0
tag = '0_25powers'

In [6]:
# Load the dictionary from the HDF5 file
target_dir = 'features'

file_path = os.path.join(target_dir, tag+'_'+cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    data[key] = np.reshape(value, (value.shape[0], -1))
    print(key, data[key].shape)

index (50, 9480)
little (50, 9480)
middle (50, 9480)
ring (50, 9480)
thumb (50, 9480)


In [7]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

In [10]:
def train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=100):
    device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device_name)
   
   
    model.to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)


    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred = torch.argmax(y_pred, dim=1)

    acc = accuracy_score(y_test.cpu(), y_pred.cpu())
    return acc

def objective(trial, train_X, test_X, train_y, test_y):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 100, 2000)
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam

    
    model = SingleLayerMLP(train_X.shape[1], hidden_size, 2, activation)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(train_X, train_y, test_X, test_y, model, criterion, optimizer, num_epochs=num_epochs)

    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    # fold_accuracies = []

    # for train_index, test_index in kf.split(X):
    #     X_train, X_test = X[train_index], X[test_index]
    #     y_train, y_test = y[train_index], y[test_index]

    #     model = SingleLayerMLP(X_train.shape[1], hidden_size, 2, activation)
    #     criterion = nn.CrossEntropyLoss()
    #     optimizer = optimizer(model.parameters(), lr=learning_rate)
    #     acc = train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=num_epochs)
    #     fold_accuracies.append(acc)

    # mean_accuracy = np.mean(fold_accuracies)
    # return mean_accuracy

def train_MLP(finger1, finger2, n_trials = 10, verbose = True):
   
    print(f'Training MLP for {finger1} vs {finger2}')

    X = np.concatenate((data[finger1], data[finger2]), axis=0)
    y = np.concatenate((np.ones(data[finger1].shape[0]), np.zeros(data[finger2].shape[0])), axis=0)

   
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, train_X, test_X, train_y, test_y), n_trials=n_trials)

    best_trial = study.best_trial

    print(f'Best trial params: {best_trial.params}')
    print(f'Best trial accuracy: {best_trial.value * 100:.2f}%')
    return best_trial.value



In [12]:
train_MLP('middle', 'ring', n_trials = 1000)

[I 2023-04-27 16:54:59,659] A new study created in memory with name: no-name-6fb38d83-110f-45dc-90d8-308b1dcbcef8


Training MLP for middle vs ring


[I 2023-04-27 16:55:00,991] Trial 0 finished with value: 0.7 and parameters: {'learning_rate': 0.0013805982068515624, 'num_epochs': 1078, 'hidden_size': 45, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.7.
[I 2023-04-27 16:55:02,527] Trial 1 finished with value: 0.7 and parameters: {'learning_rate': 4.4419473475465265e-05, 'num_epochs': 870, 'hidden_size': 75, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7.
[I 2023-04-27 16:55:04,623] Trial 2 finished with value: 0.7 and parameters: {'learning_rate': 9.629863890342874e-05, 'num_epochs': 1576, 'hidden_size': 125, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.7.
[I 2023-04-27 16:55:04,796] Trial 3 finished with value: 0.7 and parameters: {'learning_rate': 0.005528130096570195, 'num_epochs': 146, 'hidden_size': 41, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.7.
[I 2023-04-27 16:55:06,832] Trial 4 finished with value: 0.75 a

Best trial params: {'learning_rate': 0.004577309823116123, 'num_epochs': 1948, 'hidden_size': 36, 'activation': 'relu', 'optimizer': 'Adam'}
Best trial accuracy: 85.00%


0.85

In [8]:
accuracies = []
for finger1 in data:
    for finger2 in data:
        if finger1 != finger2:
            acc = train_MLP(finger1, finger2, n_trials = 10)
            accuracies.append(acc)
        else:
            break

print("#" * 50)
print(f'Mean accuracy: {np.mean(accuracies) * 100:.2f}%')
print(f'Std accuracy: {np.std(accuracies) * 100:.2f}%')
print("#" * 50)

[I 2023-04-27 16:25:02,166] A new study created in memory with name: no-name-b0d22394-09db-43f5-ac2b-f02bf6f43540


Training MLP for little vs index


[I 2023-04-27 16:25:03,470] Trial 0 finished with value: 0.45 and parameters: {'learning_rate': 0.004011571832023782, 'num_epochs': 753, 'hidden_size': 118, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.45.
[I 2023-04-27 16:25:04,949] Trial 1 finished with value: 0.5 and parameters: {'learning_rate': 0.0030613666886402783, 'num_epochs': 1981, 'hidden_size': 58, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.5.
[I 2023-04-27 16:25:05,344] Trial 2 finished with value: 0.5 and parameters: {'learning_rate': 0.0004269926037500685, 'num_epochs': 488, 'hidden_size': 94, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.5.
[I 2023-04-27 16:25:06,820] Trial 3 finished with value: 0.4 and parameters: {'learning_rate': 0.0006241663068545359, 'num_epochs': 1840, 'hidden_size': 124, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.5.
[I 2023-04-27 16:25:08,039] Trial 4 finished with value: 0.45 

Best trial params: {'learning_rate': 0.0030613666886402783, 'num_epochs': 1981, 'hidden_size': 58, 'activation': 'elu', 'optimizer': 'SGD'}
Best trial accuracy: 50.00%
Training MLP for middle vs index


[I 2023-04-27 16:25:13,427] Trial 0 finished with value: 0.4 and parameters: {'learning_rate': 9.238775287357842e-05, 'num_epochs': 544, 'hidden_size': 62, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.4.
[I 2023-04-27 16:25:14,676] Trial 1 finished with value: 0.8 and parameters: {'learning_rate': 0.042486812418228215, 'num_epochs': 1533, 'hidden_size': 124, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.8.
[I 2023-04-27 16:25:16,180] Trial 2 finished with value: 0.75 and parameters: {'learning_rate': 0.05487922092992275, 'num_epochs': 1782, 'hidden_size': 124, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.8.
[I 2023-04-27 16:25:16,772] Trial 3 finished with value: 0.8 and parameters: {'learning_rate': 0.0027250600686218084, 'num_epochs': 735, 'hidden_size': 66, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.8.
[I 2023-04-27 16:25:18,072] Trial 4 finished with value: 0.55 and paramete

...

Best trial params: {'learning_rate': 7.778643134293165e-05, 'num_epochs': 1324, 'hidden_size': 32, 'activation': 'elu', 'optimizer': 'SGD'}
Best trial accuracy: 75.00%
Training MLP for thumb vs middle


[I 2023-04-27 16:26:31,839] Trial 0 finished with value: 0.7 and parameters: {'learning_rate': 0.0010757427240546597, 'num_epochs': 1858, 'hidden_size': 16, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7.
[I 2023-04-27 16:26:33,220] Trial 1 finished with value: 0.65 and parameters: {'learning_rate': 0.000137631178587064, 'num_epochs': 1419, 'hidden_size': 56, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7.
[I 2023-04-27 16:26:33,453] Trial 2 finished with value: 0.75 and parameters: {'learning_rate': 0.0011328274519058633, 'num_epochs': 270, 'hidden_size': 103, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 2 with value: 0.75.
[I 2023-04-27 16:26:33,870] Trial 3 finished with value: 0.7 and parameters: {'learning_rate': 0.007900669120277103, 'num_epochs': 555, 'hidden_size': 74, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 2 with value: 0.75.
[I 2023-04-27 16:26:34,427] Trial 4 finished with value: 0.5 a

Best trial params: {'learning_rate': 0.0011328274519058633, 'num_epochs': 270, 'hidden_size': 103, 'activation': 'leaky_relu', 'optimizer': 'SGD'}
Best trial accuracy: 75.00%
Training MLP for thumb vs ring


[I 2023-04-27 16:26:39,434] Trial 0 finished with value: 0.9 and parameters: {'learning_rate': 0.0032293925312274264, 'num_epochs': 706, 'hidden_size': 59, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.9.
[I 2023-04-27 16:26:39,715] Trial 1 finished with value: 0.65 and parameters: {'learning_rate': 8.996804045525345e-05, 'num_epochs': 342, 'hidden_size': 22, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.9.
[I 2023-04-27 16:26:41,092] Trial 2 finished with value: 0.7 and parameters: {'learning_rate': 0.00014038475381126356, 'num_epochs': 1400, 'hidden_size': 33, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.9.
[I 2023-04-27 16:26:41,716] Trial 3 finished with value: 0.9 and parameters: {'learning_rate': 0.012722000802679266, 'num_epochs': 758, 'hidden_size': 127, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.9.
[I 2023-04-27 16:26:42,359] Trial 4 finished with value: 0.9 an

Best trial params: {'learning_rate': 0.04657124717201071, 'num_epochs': 237, 'hidden_size': 53, 'activation': 'relu', 'optimizer': 'SGD'}
Best trial accuracy: 95.00%
##################################################
Mean accuracy: 76.00%
Std accuracy: 10.91%
##################################################


In [ ]:
##################################################
# Mean accuracy: 77.50%
# Std accuracy: 11.88%
##################################################